In [31]:
import pandas as pd
import numpy as np 

In [32]:
qdf = pd.read_csv('query.csv')
qdf

,query,tag
0,what is your name?,name
1,who are you,name
2,how are you?,name
3,who created you?,create
4,who invented you?,create
5,what can you do?,news
6,what type of news you can provide?,news
7,tell me the categories of news?,news
8,do you provide live news?,news
9,when you are available?,availability


In [33]:
X = qdf['query'].tolist()
X

['what is your name?',
 'who are you',
 'how are you?',
 'who created you?',
 'who invented you?',
 'what can you do?',
 'what type of news you can provide?',
 'tell me the categories of news?',
 'do you provide live news?',
 'when you are available?',
 'can you run in offline mode?']

In [34]:
y = qdf['tag'].tolist()
y

['name',
 'name',
 'name',
 'create',
 'create',
 'news',
 'news',
 'news',
 'news',
 'availability',
 'availability']

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(X)
print(X.toarray())

[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 1]
 [0 1 0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 1 0 1 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 1 0 0]]


In [36]:
print(vectorizer.get_feature_names())

['available', 'categories', 'created', 'invented', 'live', 'mode', 'news', 'offline', 'provide', 'run', 'tell', 'type']


c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [37]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y = lb.fit_transform(y)
y

array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0]])

In [38]:
print(lb.classes_)

['availability' 'create' 'name' 'news']


In [39]:
y

array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0]])

In [40]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
model = RandomForestClassifier(random_state=1)
model.fit(X,y)

RandomForestClassifier(random_state=1)

In [42]:
text  = 'what is your name?'
input_data = vectorizer.transform([text])
print(input_data.toarray())

[[0 0 0 0 0 0 0 0 0 0 0 0]]


In [43]:
result = model.predict(input_data)

In [44]:
lb.inverse_transform(result)

array(['name'], dtype='<U12')

In [45]:
text  = 'name'
input_data = vectorizer.transform([text])
result = model.predict(input_data) # answer
print(lb.inverse_transform(result))

['name']


In [46]:
text  = 'deliver for free?'
input_data = vectorizer.transform([text])
result = model.predict(input_data) # answer
print(lb.inverse_transform(result))

['name']


In [47]:
text ="show me the menu	"
input_data = vectorizer.transform([text])
result = model.predict(input_data) # answer
print(lb.inverse_transform(result))

['name']


In [48]:
text ="are you available in lucknow?"
input_data = vectorizer.transform([text])
result = model.predict(input_data) # answer
print(lb.inverse_transform(result))

['availability']


In [49]:
text ="combo offer of the week"
input_data = vectorizer.transform([text])
result = model.predict(input_data) # answer
print(lb.inverse_transform(result))

['name']


In [50]:
text= 'are you open at 9am'
input_data = vectorizer.transform([text])
result = model.predict(input_data) # answer
ai_tag = lb.inverse_transform(result)[0]

In [51]:
rdf = pd.read_csv('response.csv')

In [52]:
resultdf = rdf[rdf['tag']==ai_tag] 
responses = resultdf.response.to_list()

In [53]:
import random

In [54]:
random.choice(responses)

'My name is News Bot 🤖'

In [55]:
import pickle

In [56]:
model_data = {
    'binarizer': lb,
    'model' : model,
    'vectorizer':vectorizer,
    'classes':lb.classes_.tolist()
}

In [57]:
with open('model.pkl','wb') as f:
    pickle.dump(model_data, f)

In [58]:
model_loaded_data = {}

with open('model.pkl','rb') as f:
    model_loaded_data = pickle.loads(f.read())

In [59]:
def predict_tag(txt='Hello world', vectorizer=None,
                model=None,binarizer=None,*args,**kwargs):
    input_vector = vectorizer.transform([txt])
    result = model.predict(input_vector)
    output_tag = lb.inverse_transform(result)
    return output_tag[0]

In [60]:
def get_bot_reply(predicted_tag):
    resultdf = rdf[rdf['tag']==predicted_tag] 
    responses = resultdf.response.to_list()
    return random.choice(responses)